In [1]:
import threading
from bybit import bybit
import pandas as pd
from datetime import datetime
import time

# Configuração da API da Bybit
client = bybit(test=False, api_key="your_api_key", api_secret="your_api_secret")

# Função para coletar dados históricos
def get_bybit_data(symbol, interval, start_time, end_time, data_list, thread_id):
    start_time_unix = int(datetime.strptime(start_time, '%Y-%m-%d').timestamp())
    end_time_unix = int(datetime.strptime(end_time, '%Y-%m-%d').timestamp())
    
    data = []

    while start_time_unix < end_time_unix:
        try:
            # Fazendo a requisição para a API
            result = client.Kline.Kline_get(symbol=symbol, interval=interval, limit=200, **{'from': start_time_unix}).result()[0]
            if 'result' not in result or len(result['result']) == 0:
                break

            # Adicionar dados obtidos à lista
            data.extend(result['result'])
            last_timestamp = result['result'][-1]['open_time']
            start_time_unix = last_timestamp + 1

        except Exception as e:
            if "429" in str(e):
                # Espera de 60 segundos apenas quando a API bloquear
                print(f"Thread {thread_id}: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...")
                time.sleep(60)
            else:
                print(f"Thread {thread_id}: Ocorreu um erro: {e}")
                break

    # Adicionar os dados coletados à lista compartilhada
    data_list.append(data)

# Função para dividir o período de tempo em partes menores (mantendo velas de 1h)
def split_time_range(start_time, end_time, num_parts):
    start_unix = int(datetime.strptime(start_time, '%Y-%m-%d').timestamp())
    end_unix = int(datetime.strptime(end_time, '%Y-%m-%d').timestamp())
    
    total_time = end_unix - start_unix
    part_size = total_time // num_parts
    times = []

    for i in range(num_parts):
        part_start = datetime.utcfromtimestamp(start_unix + i * part_size).strftime('%Y-%m-%d')
        if i == num_parts - 1:
            part_end = end_time  # O último intervalo vai até o fim do período
        else:
            part_end = datetime.utcfromtimestamp(start_unix + (i + 1) * part_size).strftime('%Y-%m-%d')
        times.append((part_start, part_end))
    
    return times

# Função principal que faz requisições em paralelo
def parallel_data_fetch(symbol, interval, start_time, end_time, num_threads=5):
    time_ranges = split_time_range(start_time, end_time, num_threads)
    threads = []
    data_list = []

    for i, (start, end) in enumerate(time_ranges):
        thread = threading.Thread(target=get_bybit_data, args=(symbol, interval, start, end, data_list, i))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    # Concatenar todos os dados coletados em um único DataFrame
    all_data = [item for sublist in data_list for item in sublist]
    df = pd.DataFrame(all_data)

    if 'open_time' in df.columns:
        df['open_time'] = pd.to_datetime(df['open_time'], unit='s')
    
    return df

# Parâmetros para buscar os dados
symbol = 'BTCUSDT'
interval = '60'
start_time = '2024-08-31'
end_time = '2024-09-01'

# Obtenção dos dados em paralelo
df_btcusdt = parallel_data_fetch(symbol, interval, start_time, end_time, num_threads=5)

# Exibir as primeiras linhas do dataframe
print(df_btcusdt.head())

# Salvar o DataFrame como CSV, se necessário
df_btcusdt.to_csv('BTCUSDT_bybit_data_1h_parallel.csv', index=False)


Thread 0: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 4: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 0: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 4: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 0: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 4: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 0: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 4: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 0: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 4: Limite de requisições atingido. Aguardando 60 segundos antes de tentar novamente...
Thread 0: Limite de requisições atingido. Aguardando 60 segu